## ebs.ErrorBudget.run_etc Required Arguments
### Fixed:  
* output_dir  
* ref_json_filename  
* pp_json_filename  
* contrast_filename  
* target_list  
* luminosity  
* eeid  
* eepsr  
* npoints 

### Possibly variable  
* exo_zodi  
* wfe  
* wfsc_factor  
* sensitivity  
* scienceInstruments  
    - QE 
    - sread  
    - idark  
    - Rs  
    - lensSamp  
    - pixelNumber
    - pixelSize
* starlightSuppressionSystems  
    - optics  
    - BW  
    - IWA  
    - core_thruput  
    - core_mean_intensity  
* observingModes  
    - SNR  

### Other  
* var_par
* *args 
* output_filename_prefix

In [1]:
import inspect
import yaml
import numpy as np
import ebs.ebs as ebs
import utilities.log_pdf as pdf

In [2]:
args_print = inspect.getfullargspec(ebs.ErrorBudget)[0]
print(
    f"ebs class arguments:\n{args_print}\n"
    )
args_print = inspect.getfullargspec(ebs.ErrorBudget.run_etc)[0]
print(
    f"ebs.run_etc method arguments:\n{args_print}"
      )

ebs class arguments:
['self', 'input_dir', 'output_dir', 'ref_json_filename', 'pp_json_filename', 'contrast_filename', 'target_list', 'luminosity', 'eeid', 'eepsr', 'exo_zodi', 'npoints']

ebs.run_etc method arguments:
['self', 'wfe', 'wfsc_factor', 'sensitivity', 'output_filename_prefix', 'var_par']


In [3]:
pdf.uniform

<function utilities.log_pdf.uniform(x, lower_bound, upper_bound)>

In [4]:
with open('mcmc.yml', 'r') as config:
    config = yaml.load(config, Loader=yaml.FullLoader)

In [5]:
# config

In [6]:
print(
    np.array(config['mcmc']['wfsc_factor']['prior_args']['lower_bound'])
)

[[9.9999e-07 9.9999e-07 9.9999e-07 9.9999e-07]
 [9.999e-05 9.999e-05 9.999e-05 9.999e-05]
 [0.00990099 0.00990099 0.00990099 0.00990099]
 [None None None None]
 [None None None None]]


In [7]:
# ErrorBudget class arguments
def pack_class_args(config=config):
    return {
        'input_dir': config['paths']['input'],
        'output_dir': config['paths']['output'],
        'ref_json_filename': config['json_files']['ref_json'],
        'pp_json_filename': config['json_files']['pp_json'],
        'contrast_filename': config['input_files']['contrast'],
        'target_list': [config['targets'][star]['HIP'] 
                       for star in config['targets']],
        'luminosity': [config['targets'][star]['luminosity']
                       for star in config['targets']],
        'eeid': [config['targets'][star]['eeid'] 
                       for star in config['targets']],
        'eepsr': [config['targets'][star]['eepsr'] 
                       for star in config['targets']],
        'exo_zodi': [config['targets'][star]['exo_zodi'] 
                       for star in config['targets']],
        'npoints': 3
            }



In [8]:
error_budget = ebs.ErrorBudget(**pack_class_args())

In [9]:
def ini_walker(config=config):
    walker_start = {}
    for key in config['mcmc'].keys():
        center = np.array(config['mcmc'][key]['ini_pars']['center'])
        spread = np.array(config['mcmc'][key]['ini_pars']['spread'])
        position = np.nan*np.ones(center.shape)
        index = np.where(center!=None)
        position[index] = (center[index] 
                + spread[index]*np.random.rand(*center.shape)[index])
        walker_start[key] = position
    return walker_start

In [10]:
print(ini_walker())

{'wfsc_factor': array([[1.00721732e-06, 1.00881691e-06, 1.00149985e-06, 1.00489158e-06],
       [1.00523944e-04, 1.00186357e-04, 1.00799113e-04, 1.00623035e-04],
       [9.92969761e-03, 9.91542186e-03, 9.96879226e-03, 9.98002647e-03],
       [           nan,            nan,            nan,            nan],
       [           nan,            nan,            nan,            nan]]), 'idark': array([1.00369712e-05])}


In [11]:
# ErrorBudget.run_etc method arguments
def pack_method_args(config=config): 
    other_args = {}
    args = inspect.getfullargspec(ebs.ErrorBudget.run_etc)[0][1:]
    for arg in args:
        if not ((arg in config['mcmc'].keys()) 
                or arg=='var_par' or arg=='*args'):
            other_args[arg] = config['run_etc_args'][arg]
    return other_args
#     wfe = np.array(config['wfe'])
#     sensitivity = np.array(config['sensitivity'])
#     output_filename_prefix = config['output_filename_prefix']

# Undefined:  wfsc_factor, var_par, *args

In [12]:
print(pack_method_args())

{'wfe': [[10.0, 10.0, 10.0, 10.0], [10.0, 10.0, 10.0, 10.0], [10.0, 10.0, 10.0, 10.0], [10.0, 10.0, 10.0, 10.0], [10.0, 10.0, 10.0, 10.0], [10.0, 10.0, 10.0, 10.0]], 'sensitivity': [[30, 30, 30, 30], [30, 30, 30, 30], [30, 30, 30, 30], [30, 30, 30, 30], [30, 30, 30, 30]], 'output_filename_prefix': 'mcmc'}


## To Do  
* Write function to read in list of variables and update ErrorBudget object accordingly  
* Write function that accepts variables and updated ErrorBudget to return exposure time
* Write function to return log probability
    